In [168]:
library(gsubfn)
model_stemp_epic <- function (NL,
         ISWWAT,
         BD,
         DLAYR,
         DS,
         DUL,
         LL,
         NLAYR,
         TAMP,
         RAIN,
         TMA,
         CUMDPT,
         DSMID,
         SW,
         TAVG,
         TMAX,
         TMIN,
         TAV,
         SRFTEMP,
         NDays,
         TDL,
         WetDay,
         ST,
         DEPIR,
         BIOMAS,
         MULCHMASS,
         SNOW){
    TBD <- 0.0
    TLL <- 0.0
    TSW <- 0.0
    X2_PREV <- 0.0
    for( L in seq(1, NLAYR + 1-1, 1)){
        TBD <- TBD + (BD[(L - 1)+1] * DLAYR[(L - 1)+1])
        TDL <- TDL + (DUL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TLL <- TLL + (LL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TSW <- TSW + (SW[(L - 1)+1] * DLAYR[(L - 1)+1])
    }
    ABD <- TBD / DS[(NLAYR - 1)+1]
    FX <- ABD / (ABD + (686.0 * exp(-(5.63 * ABD))))
    DP <- 1000.0 + (2500.0 * FX)
    WW <- 0.356 - (0.144 * ABD)
    B <- log(500.0 / DP)
    if (ISWWAT == 'Y')
    {
        PESW <- max(0.0, TSW - TLL)
    }
    else
    {
        PESW <- max(0.0, TDL - TLL)
    }
    if (NDays == 30)
    {
        for( I in seq(1, 29 + 1-1, 1)){
            WetDay[I - 1+1] <- WetDay[I + 1 - 1+1]
        }
    }
    else
    {
        NDays <- NDays + 1
    }
    if (RAIN + DEPIR > 1.E-6)
    {
        WetDay[NDays - 1+1] <- 1
    }
    else
    {
        WetDay[NDays - 1+1] <- 0
    }
    NWetDays <- sum(WetDay)
    WFT <- as.double(NWetDays) / as.double(NDays)
    CV <- (BIOMAS + MULCHMASS) / 1000.
    BCV1 <- CV / (CV + exp(5.3396 - (2.3951 * CV)))
    BCV2 <- SNOW / (SNOW + exp(2.303 - (0.2197 * SNOW)))
    BCV <- max(BCV1, BCV2)
    list[TMA, X2_PREV, ST, SRFTEMP, X2_AVG] <- SOILT_EPIC(NL, B, BCV, CUMDPT, DP, DSMID, NLAYR, PESW, TAV, TAVG, TMAX, TMIN, WetDay[NDays - 1+1], WFT, WW, TMA, X2_PREV, ST)
    return (list ("TMA"=TMA,"SRFTEMP" = SRFTEMP,"NDays" = NDays,"TDL" = TDL,"WetDay" = WetDay,"ST" = ST))
}

SOILT_EPIC <- function (NL,
         B,
         BCV,
         CUMDPT,
         DP,
         DSMID,
         NLAYR,
         PESW,
         TAV,
         TAVG,
         TMAX,
         TMIN,
         WetDay,
         WFT,
         WW,
         TMA,
         X2_PREV,
         ST){
    LAG <- 0.5
    WC <- max(0.01, PESW) / (WW * CUMDPT) * 10.0
    FX <- exp(B * ((1.0 - WC) / (1.0 + WC)) ^ 2)
    DD <- FX * DP
    if (WetDay > 0)
    {
        X2 <- WFT * (TAVG - TMIN) + TMIN
    }
    else
    {
        
        X2 <- WFT * (TMAX - TAVG) + TAVG + 2.
    }
    TMA[1] <- X2
    
    for( K in seq(5, 2, -1)){
        TMA[K] <- TMA[K - 1]
    }
    X2_AVG <- sum(TMA) / 5.0
    X3 <- (1. - BCV) * X2_AVG + (BCV * X2_PREV)
    SRFTEMP <- min(X2_AVG, X3)
    X1 <- TAV - X3
    for( L in seq(1, NLAYR + 1-1, 1)){
        ZD <- DSMID[(L - 1)+1] / DD
        F <- ZD / (ZD + exp(-.8669 - (2.0775 * ZD)))
        ST[L - 1+1] <- LAG * ST[(L - 1)+1] + ((1. - LAG) * (F * X1 + X3))
    }
    X2_PREV <- X2_AVG
    return (list ("TMA" = TMA,"X2_PREV" = X2_PREV,"ST" = ST,"SRFTEMP" = SRFTEMP,"X2_AVG" = X2_AVG))
}

init_stemp_epic <- function (NL,
         ISWWAT,
         BD,
         DLAYR,
         DS,
         DUL,
         LL,
         NLAYR,
         TAMP,
         RAIN,
         SW,
         TAVG,
         TMAX,
         TMIN,
         TAV,
         DEPIR,
         BIOMAS,
         MULCHMASS,
         SNOW){
    TMA <- array(dim=c(5,1,1))
    DSMID <- array(dim=c(NL,1,1))
    WetDay <- array(dim=c(NL,1,1))
    ST <- array(dim=c(NL,1,1))
    SWI <- array(dim=c(NL,1,1))
    SWI <- SW
    TBD <- 0.0
    TLL <- 0.0
    TSW <- 0.0
    TDL <- 0.0
    CUMDPT <- 0.0
    X2_PREV <- 0.0
    for( L in seq(1, NLAYR + 1-1, 1)){
        DSMID[L - 1+1] <- CUMDPT + (DLAYR[(L - 1)+1] * 5.0)
        CUMDPT <- CUMDPT + (DLAYR[(L - 1)+1] * 10.0)
        TBD <- TBD + (BD[(L - 1)+1] * DLAYR[(L - 1)+1])
        TLL <- TLL + (LL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TSW <- TSW + (SWI[(L - 1)+1] * DLAYR[(L - 1)+1])
        TDL <- TDL + (DUL[(L - 1)+1] * DLAYR[(L - 1)+1])
    }
    if (ISWWAT == 'Y')
    {
        PESW <- max(0.0, TSW - TLL)
    }
    else
    {
        PESW <- max(0.0, TDL - TLL)
    }
    ABD <- TBD / DS[(NLAYR - 1)+1]
    FX <- ABD / (ABD + (686.0 * exp(-(5.63 * ABD))))
    DP <- 1000.0 + (2500.0 * FX)
    WW <- 0.356 - (0.144 * ABD)
    B <- log(500.0 / DP)
    for( I in seq(1, 5 + 1-1, 1)){
        TMA[I - 1+1] <- as.integer(TAVG * 10000.) / 10000.
    }
    X2_AVG <- TMA[(1 - 1)+1] * 5.0
    for( L in seq(1, NLAYR + 1-1, 1)){
        ST[L - 1+1] <- TAVG
    }
    WFT <- 0.1
    WetDay <-  array(c(0), dim=c(30,1,1))
    NDays <- 0
    CV <- MULCHMASS / 1000.
    BCV1 <- CV / (CV + exp(5.3396 - (2.3951 * CV)))
    BCV2 <- SNOW / (SNOW + exp(2.303 - (0.2197 * SNOW)))
    BCV <- max(BCV1, BCV2)
    for( I in seq(1, 8 + 1-1, 1)){
        list[TMA, X2_PREV, ST, SRFTEMP, X2_AVG] <- SOILT_EPIC(NL, B, BCV, CUMDPT, DP, DSMID, NLAYR, PESW, TAV, TAVG, TMAX, TMIN, 0, WFT, WW, TMA, X2_PREV, ST)
    }
    return (list ("TMA" = TMA,"CUMDPT" = CUMDPT,"DSMID" = DSMID,"SRFTEMP" = SRFTEMP,"NDays" = NDays,"TDL" = TDL,"WetDay" = WetDay,"ST" = ST))
}

In [169]:
library(gsubfn)
DYNAMIC = 1
    ISWWAT = 'Y'
    BD    = array(c(1.6),dim=c(4,1,1))
    DLAYR = array(c(10.0),dim=c(4,1,1))
    DS = array(c(0.0),dim=c(4,1,1))
    DS[1] =10.0
    DS[2] =20.0
    DS[3] =30.0
    DS[4] =40.0
    DUL   = array(c(0.3),dim=c(20,1,1))
    LL    = array(c(0.2),dim=c(20,1,1))
    NLAYR = 4
    MSALB = 0.13
    SRAD    = 20.0
    SW      = array(c(0.2),dim=c(20,1,1))
    TAVG    = 25.0
    TMAX    = 30.0
    TMIN    = 20.0
    XLAT    = 28.0
    TAV     = 20.0
    TAMP    = 10.0
    RAIN = 0.0
    MULCHMASS = 0.0
    SNOW = 0.0
    DEPIR = 0.0
    BIOMAS = 0.0
    NL = 20

    list [ TMA,CUMDPT, DSMID,SRFTEMP, NDays,TDL, WetDay, ST]= init_stemp_epic(NL,ISWWAT,BD,DLAYR, DS,DUL,LL,NLAYR,TAMP,RAIN,SW,TAVG,TMAX,TMIN,TAV, DEPIR,BIOMAS,MULCHMASS,SNOW)
for (k in seq(1,299)){
list [TMA,SRFTEMP,NDays,TDL, WetDay,ST]= model_stemp_epic(NL,ISWWAT, BD, DLAYR,DS,DUL,LL, NLAYR,TAMP, RAIN,TMA,CUMDPT,DSMID, SW,TAVG,TMAX,TMIN, TAV,SRFTEMP, NDays,TDL, WetDay,ST,DEPIR,BIOMAS, MULCHMASS,SNOW)
}
print(ST[1:4])



[1] 25.43649 23.04468 21.64317 20.88921


In [42]:
z

$TMA
, , 1

     [,1]
[1,] 27.5
[2,] 27.5
[3,] 27.5
[4,] 27.5
[5,] 27.5


$CUMDPT
[1] 400

$DSMID
, , 1

      [,1]
 [1,]   50
 [2,]  150
 [3,]  250
 [4,]  350
 [5,]   NA
 [6,]   NA
 [7,]   NA
 [8,]   NA
 [9,]   NA
[10,]   NA
[11,]   NA
[12,]   NA
[13,]   NA
[14,]   NA
[15,]   NA
[16,]   NA
[17,]   NA
[18,]   NA
[19,]   NA
[20,]   NA


$SRFTEMP
[1] 27.5

$NDays
[1] 0

$TDL
[1] 12

$WetDay
, , 1

      [,1]
 [1,]    0
 [2,]    0
 [3,]    0
 [4,]    0
 [5,]    0
 [6,]    0
 [7,]    0
 [8,]    0
 [9,]    0
[10,]    0
[11,]    0
[12,]    0
[13,]    0
[14,]    0
[15,]    0
[16,]    0
[17,]    0
[18,]    0
[19,]    0
[20,]    0
[21,]    0
[22,]    0
[23,]    0
[24,]    0
[25,]    0
[26,]    0
[27,]    0
[28,]    0
[29,]    0
[30,]    0


$ST
, , 1

          [,1]
 [1,] 25.80490
 [2,] 23.25960
 [3,] 21.76815
 [4,] 20.96581
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA
[13,]       NA
[14,]       NA
[15,]       NA
[16,]       NA
[17,]       NA
[18,]       NA
[19,]       NA
[20,]       NA

In [63]:
 WFT = 0.1256
    TAVG    = 25.0
    TMAX    = 30.0

X2 <- WFT * (TMAX - TAVG) + TAVG + 2.

In [64]:
X2

[1] 27.628

In [20]:
test <- function (a,
         b){
    for( j in seq(5, 2+1, -1)){
        a <- a + 10
    }
    return( a)
}

In [21]:
test(1,0)

[1] 31

In [72]:
a = array(c(1,2,3), dim=c(3,1,1))

In [74]:
a[1]

[1] 1

In [80]:
a[1]=25


In [81]:
a

, , 1

     [,1]
[1,]   25
[2,]    2
[3,]    3

In [156]:
library(gsubfn)
test1 <- function(a, b){
    
    addi = a+b
    sous= a-b
    
    return (list("j"=addi,"k"=sous))
    
}

In [ ]:
test1 <- function(a, b){
    
    mult = a*b
    div= a/b
    return (list("j"=mult,"k"=div))
    
}

In [157]:
j = 3
k = 1
for (s in seq(1,3,1)){
    print(c(s,j,k))
    list[j,k] = test1(j,k)
    print(c(j,k))
}

[1] 1 3 1
[1] 4 2
[1] 2 4 2
[1] 6 2
[1] 3 6 2
[1] 8 4


In [130]:
print(c(j,k))

[1] 93 NA


In [150]:
a=5
b=3

In [153]:
list[j,k] = test1(a,b)

In [154]:
k

[1] 2